In [122]:
import ollama
import pandas as pd
import sqlite3
import re
from collections import Counter
import json
from thefuzz import fuzz
from itertools import combinations

In [ ]:
# Retrieve recipes data from db
conn = sqlite3.connect("./data/all_recipes.db")
query = "SELECT * FROM recipes"
df_recipes = pd.read_sql_query(query, conn)
conn.close()

In [3]:
# Function to find all unique words in a column regaerdless of the entry being a list of words
def find_unique_vals(df, column_name):
    unique_list = set(
        word.strip()
        for courses in df[column_name]
        if courses is not None  # Skip None values
        for word in courses.split(";")
    )
    return unique_list

In [ ]:
# Find all ingredients mentioned across all recipes
ingredients_list = find_unique_vals(df_recipes, "ingredients")
print("Number of ingredients:", len(ingredients_list))

# Convert from set to list
ingredients_list = list(ingredients_list)

ingredient_mappings = dict()

Number of ingredients: 3617


In [6]:
# Function to clean non-alphabetical characters (except spaces)
def clean_ingredient(ingredient):
    # Remove anything that is not a letter or space
    return re.sub(r"[^a-zA-Z\s]", "", ingredient)


# Clean the entire ingredients list before processing
ingredients_list = [clean_ingredient(ingredient) for ingredient in ingredients_list]

In [ ]:
print("Number of ingredients to process: ", len(ingredients_list))

# Updated regex pattern to support both single and double quotes
pattern = r"['\"]([a-zA-Z\s]+)['\"]\s*:\s*['\"]([a-zA-Z\s]+)['\"]"

# how many ingredients to give the LLM at a time
n = 20


while ingredients_list:

    # gets the current batch of ingredients for LLM to process
    current_ingredients, ingredients_list = ingredients_list[:n], ingredients_list[n:]
    number_ingredients = len(current_ingredients)

    content = f"""The following are common ingredient variations. Normalize each ingredient variation by mapping it to the appropriate standardized name:

    Salt, Kosher Salt, Sea Salt, Flaky Salt -> Salt
    Olive Oil, Extra Virgin Olive Oil, Virgin Olive Oil -> Olive Oil
    Tomatoes, Canned Tomatoes, Cherry Tomatoes, Roma Tomatoes -> Tomatoes
    Potatoes, Russet Potatoes, Yukon Gold Potatoes -> Potatoes
    Garlic, Garlic Powder, Minced Garlic -> Garlic

    Create a python dictionary, where each key refers to one of the original ingredients lists below and its value is your answer. You can only give one ingredient answer. Do not include the above examples in the python dictionary to be created. Do not return anything other than the python dictionary in your response.

    There are {number_ingredients} listed below, as such you must output {number_ingredients} ingredient mappings. If the ingredient is in fact not an ingredient then assign it 'NaN'.
    Normalize the ingredients found in this python list:
    {current_ingredients}"""

    response = ollama.generate(model="gemma2", prompt=content)

    #### ADD INGREDIENT MAPPINGS TO GLOBAL LIST
    raw_output = response["response"]

    # Find all matching pairs
    matches = re.findall(pattern, raw_output)

    # Create a dictionary from the matches
    ingredient_standardization = dict(matches)
    ingredient_mappings.update(ingredient_standardization)

    ## Find all the ingredients the LLM didn't process so it can re-process it
    llm_returned_keys = list(ingredient_standardization.keys())
    missing_items = [
        item for item in current_ingredients if item not in llm_returned_keys
    ]
    ingredients_list.extend(missing_items)

    print(f"\rNumber of ingredients processed: {len(ingredient_mappings)}", end="")

    if ingredients_list == []:
        break

Number of ingredients to process:  3597
Number of ingredients processed: 3601

KeyboardInterrupt: 

In [ ]:
# Saves standadised names to a .txt file so its not lost
with open("standardised_names.txt", "w") as convert_file:
    convert_file.write(json.dumps(ingredient_mappings))

------

## Re-Evaluate Ingredients with NaN Values

In [ ]:
# Get all keys where the values are NaN
nan_ingredients = [key for key, value in ingredient_mappings.items() if value == "NaN"]
print(nan_ingredients)
print(len(nan_ingredients))

# Saves NaN names to a .txt file so its not lost
with open("NaN_names.txt", "w") as convert_file:
    convert_file.write(json.dumps(nan_ingredients))

['Homemade Shrimp Tempura', 'FoodSaver FM Vacuum Sealing System', 'Optional Garnish', 'Reynolds Wrap NonStick Foil', 'pine nutsdried jujubescitron syrupmint leaves', 'leftover rice water', 'reserved tomato blanching water', 'Chinese', 'udon noodles total oz   g', 'reserved seafood sauce', 'burger buns', 'water for stove top', 'astragalus', 'sev farsan', 'Syrup to taste', 'shio koji karaage', 'Yaki Onigiri Grilled Rice Ball', 'warm water F  C', 'Homemade Castella Cake', 'cold tap water', 'flat plate or smooth metal or stone surface', 'dry rub of your choice', 'Chrysanthemum leaves or Crown Daisies', 'block', 'liquor of your choice', 'reserved shiitake soaking liquid', 'mixed fruit or vanilla essence', 'gyoza', 'whole bonein', 'reserved water from soaking kiriboshi daikon', 'soboro don', 'lily bulb', 'codonopsis root', 'Lamb with carrot and zucchini', 'SPAM', 'homemade kansui', 'rosdee flavor seasoning', 'sev or bhujia', 'Chopped walnuts and  or raw sugar', 'suki sauce', 'vegetables of y

In [ ]:
print("Number of ingredients to process: ", len(nan_ingredients))

# Updated regex pattern to support both single and double quotes
pattern = r"['\"]([a-zA-Z\s]+)['\"]\s*:\s*['\"]([a-zA-Z\s]+)['\"]"

# how many ingredients to give the LLM at a time
n = 20

counter = 1

nan_ingredient_mappings = dict()

while nan_ingredients:

    # gets the current batch of ingredients for LLM to process
    current_ingredients, nan_ingredients = nan_ingredients[:n], nan_ingredients[n:]
    number_ingredients = len(current_ingredients)

    content = f"""The following are common ingredient variations. Normalize each ingredient variation by mapping it to the appropriate standardized name:

    Salt, Kosher Salt, Sea Salt, Flaky Salt -> Salt
    Olive Oil, Extra Virgin Olive Oil, Virgin Olive Oil -> Olive Oil
    Tomatoes, Canned Tomatoes, Cherry Tomatoes, Roma Tomatoes -> Tomatoes
    Potatoes, Russet Potatoes, Yukon Gold Potatoes -> Potatoes
    Garlic, Garlic Powder, Minced Garlic -> Garlic

    Create a python dictionary, where each key refers to one of the original ingredients lists below and its value is your answer. You can only give one ingredient answer. Do not include the above examples in the python dictionary to be created. Do not return anything other than the python dictionary in your response.

    There are {number_ingredients} listed below, as such you must output {number_ingredients} ingredient mappings. If the ingredient is in fact not an ingredient then assign it 'NaN'.
    Normalize the ingredients found in this python list:
    {current_ingredients}"""

    response = ollama.generate(model="gemma2", prompt=content)

    #### ADD INGREDIENT MAPPINGS TO GLOBAL LIST
    raw_output = response["response"]

    # Find all matching pairs
    matches = re.findall(pattern, raw_output)

    # Create a dictionary from the matches
    ingredient_standardization = dict(matches)
    nan_ingredient_mappings.update(ingredient_standardization)

    ## Find all the ingredients the LLM didn't process so it can re-process it
    llm_returned_keys = list(ingredient_standardization.keys())
    missing_items = [
        item for item in current_ingredients if item not in llm_returned_keys
    ]
    nan_ingredients.extend(missing_items)

    print(
        f"\rCycle: {counter}, Number of ingredients processed: {len(nan_ingredient_mappings)}, Number left to process: {len(nan_ingredients)}",
        end="",
    )
    counter += 1

    if nan_ingredients == []:
        break

Number of ingredients to process:  332
Cycle: 18, Number of ingredients processed: 337, Number left to process: 020

In [102]:
nan_ingredient_mappings

{'Homemade Shrimp Tempura': 'Shrimp',
 'FoodSaver FM Vacuum Sealing System': 'NaN',
 'Optional Garnish': 'NaN',
 'Reynolds Wrap NonStick Foil': 'NaN',
 'leftover rice water': 'Rice Water',
 'reserved tomato blanching water': 'Tomato Water',
 'Chinese': 'NaN',
 'udon noodles total oz   g': 'Udon Noodles',
 'reserved seafood sauce': 'Seafood Sauce',
 'burger buns': 'Buns',
 'water for stove top': 'Water',
 'astragalus': 'Astragalus',
 'sev farsan': 'Sev Farsan',
 'Syrup to taste': 'Syrup',
 'shio koji karaage': 'Chicken',
 'Yaki Onigiri Grilled Rice Ball': 'Rice',
 'warm water F  C': 'Water',
 'Homemade Castella Cake': 'Cake',
 'cold tap water': 'Water',
 'flat plate or smooth metal or stone surface': 'NaN',
 'dry rub of your choice': 'NaN',
 'Chrysanthemum leaves or Crown Daisies': 'NaN',
 'block': 'NaN',
 'liquor of your choice': 'NaN',
 'reserved shiitake soaking liquid': 'NaN',
 'mixed fruit or vanilla essence': 'NaN',
 'gyoza': 'NaN',
 'whole bonein': 'NaN',
 'reserved water from so

In [ ]:
len(nan_ingredient_mappings)

# Get original ingredients that were NaN
nan_ingredients = [key for key, value in ingredient_mappings.items() if value == "NaN"]

# Manually check the ingredients originally listed as nan as their new stadnadised names
for i in range(len(nan_ingredients)):
    current = nan_ingredients[i]
    standardised = nan_ingredient_mappings[current]
    answer = input(f"[{i}/{len(nan_ingredients)}] {current} --> {standardised}: ")
    if answer == "":
        ingredient_mappings[current] = standardised
    else:
        ingredient_mappings[current] = answer

In [149]:
# Saves standadised names to a .txt file so its not lost
with open("standardised_names.txt", "w") as convert_file:
    convert_file.write(json.dumps(ingredient_mappings))

------

## Group Standardised Name into Categories taking into account for mispellings and pluralisation

In [769]:
# Create ingredient maps where key is the standard name and values are lists of ingreidnets that map to it
mapping_lists = dict()

for key, value in ingredient_mappings.items():
    if value not in mapping_lists:
        mapping_lists[value] = [key]
    else:
        mapping_lists[value].append(key)

print(sorted(mapping_lists.keys()))

['Aamchoor', 'Aamchur', 'Abalone', 'Acorn Powder', 'Active Dry Yeast', 'Agar Agar', 'Agave Syrup', 'Ajwain', 'Allspice', 'Almond', 'Almond Extract', 'Almond Meal', 'Almond Milk', 'Almonds', 'Amchur Powder', 'Anaheim Pepper', 'Anaheim peppers', 'Anchovies', 'Anchovy', 'Anchovy Sauce', 'Anchovy Stock', 'Apple', 'Apple Cider Vinegar', 'Apple Juice', 'Apricots', 'Arrowroot', 'Artichoke Heart', 'Arugula', 'Asafetida', 'Asafoetida', 'Asparagus', 'Astragalus', 'Avocado', 'Avocado Oil', 'Bacon', 'Baguette', 'Baking Powder', 'Baking Soda', 'Bamboo Leaves', 'Bamboo Shoot', 'Bamboo Shoots', 'Banana', 'Banana Leaves', 'Barbecue Sauce', 'Barley', 'Basil', 'Basil Seeds', 'Basmati Rice', 'Bay Leaf', 'Bay Leaves', 'Bean Liquid', 'Bean Paste', 'Bean Sauce', 'Bean Sprout', 'Bean Sprouts', 'Beans', 'Beef', 'Beef Bones', 'Beef Bowl', 'Beef Brisket', 'Beef Broth', 'Beef Chuck Roast', 'Beef Short Ribs', 'Beef Stock', 'Beef Tongue', 'Beer', 'Beetroot', 'Beets', 'Bell Pepper', 'Bell Peppers', 'Bellflower Root

In [743]:
keys_standardised_names = list(mapping_lists.keys())
print(len(keys_standardised_names))

# Initialize an empty dictionary to store the fuzzy scores
fuzzy_scores = {}


# Loop over each pair of ingredients using itertools.combinations to avoid duplicates
for key1, key2 in combinations(set(keys_standardised_names), 2):
    # Compute the fuzzy match score
    score = fuzz.ratio(key1, key2)

    # Store the score in the dictionary
    if (key2, key1) not in fuzzy_scores:
        fuzzy_scores[(key1, key2)] = score

list_fuzz_pairs = []

# Display the fuzzy scores
for pair, score in fuzzy_scores.items():
    if score >= 75:
        pair = list(pair)
        list_fuzz_pairs.append(pair)

print(list_fuzz_pairs)
print(len(list_fuzz_pairs))

924
[['Bouillon Cube', 'Bouillon'], ['Peanuts', 'Peanut'], ['Peanuts', 'Pecans'], ['Asafetida', 'Asafoetida'], ['Lemon Zest', 'Lemons'], ['Yellow Curry Powder', 'Curry Powder'], ['Takoyaki Sauce', 'Teriyaki Sauce'], ['Takoyaki Sauce', 'Okonomiyaki Sauce'], ['Cauliflower Rice', 'Cauliflower'], ['Apple Juice', 'Pineapple Juice'], ['Okara', 'Okra'], ['Lily Flower Bud', 'Lily Flower'], ['Lily Flower Bud', 'Lily Flowers'], ['Mushroom Powder', 'Mushroom Water'], ['Peppercorns', 'Peppers'], ['Peppercorns', 'Peppercorn'], ['Pickled Mustard Greens', 'Mustard Greens'], ['Pickled Mustard Greens', 'Chinese Pickled Mustard Greens'], ['Rice Powder', 'Rice Paper'], ['Chilies', 'Chives'], ['Chilies', 'Chiles'], ['Chilies', 'Chili'], ['Pickled Vegetables', 'Mixed Vegetables'], ['Pickled Vegetables', 'Pickled Leaves'], ['Coconut Milk', 'Coconut Oil'], ['Cashew', 'Cashews'], ['Wood Ear Mushroom', 'Wood Ear Mushrooms'], ['Shiitake Mushrooms', 'Shimeji Mushrooms'], ['Shiitake Mushrooms', 'Straw Mushrooms']

In [ ]:
def get_master_name(ingredient, merge_history):
    # This function will return the final master ingredient after all merges.
    while ingredient in merge_history:
        ingredient = merge_history[ingredient]
    return ingredient


# Track the final master names after merges, where key is the ingredient and value is its new name
merge_history = dict()

# Attempt to merge the ingredients
for i in range(len(list_fuzz_pairs)):
    ingredient1, ingredient2 = list_fuzz_pairs[i]

    # Get the master names for both ingredients
    ingredient1_master = get_master_name(ingredient1, merge_history)
    ingredient2_master = get_master_name(ingredient2, merge_history)

    # If both ingredients already have the same master name, no merge is needed
    if ingredient1_master == ingredient2_master:
        print(
            f"Ingredients '{ingredient1}' and '{ingredient2}' are already merged as '{ingredient1_master}'. Skipping merge."
        )

    # Otherwise, ask for human input
    else:
        user_input = input(
            f"[{i}/{len(list_fuzz_pairs)}] 1: {ingredient1_master}, 2: {ingredient2_master}"
        )

        if user_input == "1":
            # Merge ingredient2 into ingredient1
            mapping_lists[ingredient1_master].extend(mapping_lists[ingredient2_master])
            del mapping_lists[ingredient2_master]
            merge_history[ingredient2_master] = ingredient1_master
        elif user_input == "2":
            # Merge ingredient1 into ingredient2
            mapping_lists[ingredient2_master].extend(mapping_lists[ingredient1_master])
            del mapping_lists[ingredient1_master]
            merge_history[ingredient1_master] = ingredient2_master

Ingredients 'Lily Flower' and 'Lily Flowers' are already merged as 'Lily Flower'. Skipping merge.
Ingredients 'Chili Peppers' and 'Green Chili Pepper' are already merged as 'Chili Pepper'. Skipping merge.
Ingredients 'Chili Peppers' and 'Chilli Peppers' are already merged as 'Chili Pepper'. Skipping merge.
Ingredients 'Chili Peppers' and 'Chili Pepper' are already merged as 'Chili Pepper'. Skipping merge.
Ingredients 'Chili Peppers' and 'Chile Pepper' are already merged as 'Chili Pepper'. Skipping merge.
Ingredients 'Chili Peppers' and 'Chilli Pepper' are already merged as 'Chili Pepper'. Skipping merge.
Ingredients 'Chili Peppers' and 'Chile Peppers' are already merged as 'Chili Pepper'. Skipping merge.
Ingredients 'Green Chili Pepper' and 'Chilli Peppers' are already merged as 'Chili Pepper'. Skipping merge.
Ingredients 'Green Chili Pepper' and 'Chili Pepper' are already merged as 'Chili Pepper'. Skipping merge.
Ingredients 'Green Chili Pepper' and 'Green Chillies' are already merged

In [772]:
len(mapping_lists)

754

In [ ]:
# If mapping lists has been re-initialised and merge history has already been created
for key, value in merge_history.items():
    master_ingredient = get_master_name(key, merge_history)
    mapping_lists[master_ingredient].extend(mapping_lists[key])
    del mapping_lists[key]

In [ ]:
# Convert the list to a DataFrame and save it to a CSV
keys_standardised_names = list(mapping_lists.keys())
df = pd.DataFrame(sorted(keys_standardised_names), columns=["Ingredients"])

# Save the DataFrame to a CSV file
csv_file_path = "ingredients_for_manual_cleaning.csv"
df.to_csv(csv_file_path, index=False)

In [847]:
len(mapping_lists)

754

In [ ]:
# Load the CSV file
file_path = "manually_cleaned_done.csv"
df = pd.read_csv(file_path)

# Loop over each row in the DataFrame
for index, row in df.iterrows():
    master_ingredient = row["Ingredients"]  # This is the standardized name
    variations = row.dropna()[
        1:
    ].tolist()  # The variations are in the following columns

    if master_ingredient not in mapping_lists:
        mapping_lists[master_ingredient] = []

    for item in variations:
        # Append all variations to the master ingredient's list
        mapping_lists[master_ingredient].extend(mapping_lists[item])
        del mapping_lists[item]

# Output the resulting mapping lists
mapping_lists

{'Chili Pepper': ['ichimi togarashi Japanese chili pepper',
  'small green chili',
  'dried chili pepper',
  'dried red chilies',
  'cubed chili pepper',
  'Thai chili pepper or serrano pepper chopped',
  'Thai chili pepper',
  'Green Chili Peppers',
  'dry Kashmiri red chilies',
  'green chili pepper',
  'jalapeno or Thai chili',
  'fresh red chilies',
  'dried red chili',
  'Thai chilis',
  'Sliced hot chili pepper for garnish',
  'prik chi fa chili',
  'serrano peppers',
  'gochukaru',
  'Chinese long chili pepper',
  'whole dried chili peppers',
  'fresh red or green chili pepper',
  'green chilies or jalapeo',
  'green chilis',
  'bird eye chili pepper',
  'dried whole red chilies',
  'chilies to taste',
  'Thai chilies',
  'Dried chili peppers',
  'dried Chinese chili peppers',
  'red chilies',
  'fresh Thai chili peppers',
  'Thinly sliced chili peppers',
  'thai red chili',
  'red chili pepper',
  'red chilis',
  'red chili',
  'red spur chili',
  'fresh red chilis',
  'birds e

----

In [933]:
def clean_ingredients_in_string(ingredient):
    return re.sub(r"[^a-zA-Z\s]", "", ingredient).strip()


# Clean the entire ingredients list before processing
df_recipes["cleaned_ingredients"] = df_recipes["ingredients"].apply(
    lambda x: [clean_ingredients_in_string(ingredient) for ingredient in x.split(";")]
)


# Normalize ingredients based on the mapping dictionary
def normalize_ingredient(ingredient):
    for standard_name, variations in mapping_lists.items():
        if ingredient in [var for var in variations]:
            return standard_name
    return "NaN"  # Return NaN as some ingredients may have failed going through the LLM or the key changed and it cant be found


# Create a new column 'normalised_ingredients' based on the mapping
df_recipes["normalised_ingredients"] = df_recipes["cleaned_ingredients"].apply(
    lambda x: [normalize_ingredient(ingredient) for ingredient in x]
)

In [934]:
# Count the number of rows where 'NaN' is found in the 'normalised_ingredients' column
nan_count = df_recipes[
    df_recipes["normalised_ingredients"].apply(lambda x: "NaN" in str(x))
].shape[0]

print("Number of rows with a NaN value in the normalised ingredients list:", nan_count)

Number of rows with a NaN value in the normalised ingredients list: 290


In [935]:
# Filter rows where 'normalised_ingredients' does not contain 'NaN'
df_no_nan = df_recipes[
    df_recipes["normalised_ingredients"].apply(lambda x: "NaN" not in str(x))
].reset_index()

In [936]:
# Updated function to handle days, hours, and minutes (including plural forms)
def convert_to_minutes_extended(time_str):
    # Handle None or NaN values
    if not isinstance(time_str, str):
        return None  # Or 0, if you want to return 0 minutes for invalid entries

    time_str = time_str.lower()
    days = 0
    hours = 0
    minutes = 0

    # Extract days
    if "day" in time_str:
        days_match = re.search(r"(\d+)\s*day", time_str)
        if days_match:
            days = int(days_match.group(1))

    # Extract hours
    if "hour" in time_str:
        hours_match = re.search(r"(\d+)\s*hour", time_str)
        if hours_match:
            hours = int(hours_match.group(1))

    # Extract minutes
    if "minute" in time_str:
        minutes_match = re.search(r"(\d+)\s*minute", time_str)
        if minutes_match:
            minutes = int(minutes_match.group(1))

    # Convert to total minutes
    return days * 1440 + hours * 60 + minutes  # 1 day = 1440 minutes


# Apply the updated function to the total_time column
df_no_nan["total_time_minutes"] = df_no_nan["total_time"].apply(
    convert_to_minutes_extended
)

df_no_nan = df_no_nan.drop("image_data", axis=1)
df_no_nan = df_no_nan.drop("index", axis=1)
df_no_nan = df_no_nan.drop("cleaned_ingredients", axis=1)


df_no_nan

,title,link,image_url,description,total_time,course,cuisine,ingredients,calories,normalised_ingredients,total_time_minutes
0,Japanese Milk Bread (Shokupan),https://www.justonecookbook.com/japanese-milk-...,https://www.justonecookbook.com/wp-content/upl...,Japanese Milk Bread is possibly the best versi...,3 hours,Breakfast,Japanese,warm water; sugar; Diamond Crystal kosher salt...,1645 kcal,"[Water, Sugar, Salt, Honey, Yeast, Flour, Milk...",180.0
1,Chicken Chashu,https://www.justonecookbook.com/chicken-chashu/,https://www.justonecookbook.com/wp-content/upl...,Juicy and tender Chicken Chashu is a lighter v...,1 hour,Main Course,Japanese,"boneless, skin-on chicken thighs; green onions...",None,"[Chicken, Green Onions, Ginger, Oil, Water, Co...",60.0
2,Gyudon (Japanese Beef Rice Bowl),https://www.justonecookbook.com/gyudon/,https://www.justonecookbook.com/wp-content/upl...,With thinly sliced beef and tender onions simm...,20 minutes,Main Course,Japanese,onion; green onion/scallion; thinly sliced bee...,657 kcal,"[Onions, Green Onions, Beef, Stock, Sake, Cook...",20.0
3,Japanese Beef Curry,https://www.justonecookbook.com/japanese-beef-...,https://www.justonecookbook.com/wp-content/upl...,"With tender chunks of beef, potatoes, carrots,...",3 hours,Main Course,Japanese,onions; unsalted butter; neutral oil; russet p...,426 kcal,"[Onions, Butter, Oil, Potatoes, Carrots, Mushr...",180.0
4,Japanese Cheesecake,https://www.justonecookbook.com/souffle-japane...,https://www.justonecookbook.com/wp-content/upl...,Jiggly and fluffy Japanese Cheesecake is a cro...,1 hour,Dessert,Japanese,unsalted butter; large eggs (50 g each w/o she...,3560 kcal,"[Butter, Eggs, Cream Cheese, Butter, Cream, Su...",60.0
...,...,...,...,...,...,...,...,...,...,...,...
1828,Thai Vegetable Soup (Tom Jued),https://hungryinthailand.com/thai-vegetable-so...,https://hungryinthailand.com/wp-content/upload...,Tom jued is a healthy Thai vegetable soup reci...,40 minutes,"Main Course, Soup",Thai,garlic; coriander root; black peppercorns; bou...,63 kcal,"[Garlic, Coriander, Black Pepper, Stock, Oyste...",40.0
1829,Tom Kha Gai Recipe (Thai Coconut Chicken Soup),https://hungryinthailand.com/tom-kha-gai/,https://hungryinthailand.com/wp-content/upload...,Try this tom kha gai recipe—a healthy and quic...,20 minutes,"Main Course, Soup",Thai,water; coconut milk; lemongrass; galangal; kaf...,451 kcal,"[Water, Coconut Milk, Lemongrass, Galangal, Li...",20.0
1830,Tom Yum Kung Recipe (Spicy Thai Shrimp Soup),https://hungryinthailand.com/tom-yum-kung/,https://hungryinthailand.com/wp-content/upload...,This tom yum kung recipe is an authentic Thai ...,25 minutes,"Main Course, Soup",Thai,shrimp; Asian mushrooms; lemongrass; shallots;...,198 kcal,"[Shrimps, Mushrooms, Lemongrass, Shallots, Lim...",25.0
1831,Thai Chicken Glass Noodle Soup Recipe,https://hungryinthailand.com/thai-chicken-glas...,https://hungryinthailand.com/wp-content/upload...,Enjoy a quick and easy Thai chicken glass nood...,30 minutes,Soup,Thai,glass noodles; dried chilies; chilies; shallot...,644 kcal,"[Noodles, Chili Pepper, Chili Pepper, Shallots...",30.0


In [ ]:
def count_unique_vals(df, column_name):
    # Flatten all words into a single list
    all_ingredients = [
        ingredient.strip()  # Strip any extra spaces from ingredients
        for ingredients in df[column_name]  # Iterate over the rows in the column
        for ingredient in ingredients  # Iterate over each ingredient in the list
        if ingredient is not None  # Skip None values
    ]

    # Count occurrences of each unique ingredient
    ingredient_counts = Counter(all_ingredients)

    return ingredient_counts

In [906]:
ingredient_count = dict(count_unique_vals(df_no_nan, "normalised_ingredients"))

names = ingredient_count.keys()

# Sort the dictionary by values in descending order
sorted_ingredient_counts = dict(
    sorted(ingredient_count.items(), key=lambda item: item[1], reverse=True)
)

# Display the sorted result
sorted_ingredient_counts

{'Salt': 1417,
 'Soy Sauce': 1266,
 'Sugar': 1261,
 'Water': 916,
 'Garlic': 797,
 'Oil': 710,
 'Ginger': 700,
 'Cooking Wine': 687,
 'Cornstarch': 613,
 'Onions': 584,
 'Green Onions': 519,
 'Peanut Oil': 449,
 'Eggs': 409,
 'Sesame Oil': 365,
 'Vinegar': 349,
 'Black Pepper': 319,
 'Flour': 315,
 'Chicken': 306,
 'Chili Pepper': 283,
 'Sake': 278,
 'Carrots': 271,
 'Coriander': 263,
 'Rice': 258,
 'Sesame Seeds': 250,
 'Stock': 247,
 'Pork': 247,
 'Mushrooms': 245,
 'Oyster Sauce': 225,
 'Butter': 223,
 'White Pepper': 182,
 'Tofu': 178,
 'Broth': 172,
 'Noodles': 172,
 'Tomatoes': 134,
 'Shrimps': 124,
 'Fish Sauce': 117,
 'Turmeric': 116,
 'Milk': 114,
 'Bean Paste': 114,
 'Peppers': 113,
 'Lemons': 112,
 'Cabbage': 111,
 'Beef': 110,
 'Chili Powder': 108,
 'Sichuan Peppercorns': 105,
 'Cumin': 104,
 'Eggplant': 99,
 'Miso Paste': 96,
 'Chili Oil': 91,
 'Radish': 90,
 'Cucumbers': 80,
 'Coconut Milk': 79,
 'Broccoli': 76,
 'Rice Flour': 75,
 'Honey': 72,
 'Bok Choy': 72,
 'Chili Pe

In [937]:
# Convert lists to strings
if "normalised_ingredients" in df_no_nan.columns:
    df_no_nan["normalised_ingredients"] = df_no_nan["normalised_ingredients"].apply(
        lambda x: "; ".join(x) if isinstance(x, list) else x
    )

if "cleaned_ingredients" in df_no_nan.columns:
    df_no_nan["cleaned_ingredients"] = df_no_nan["cleaned_ingredients"].apply(
        lambda x: "; ".join(x) if isinstance(x, list) else x
    )

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect("standardised_recipes.db")

# Save DataFrame to SQL database
df_no_nan.to_sql("recipes", conn, if_exists="replace", index=False)

# Close the connection
conn.close()

print("Data saved to database!")

Data saved to database!
